In [64]:
from functions.biref_fisher_class import FisherBiref
from pspy import so_dict
import numpy as np
from math import pi
import yaml
from copy import deepcopy

spectra_pspy = ["TT", "TE", "TB", "ET", "BT", "EE", "EB", "BE", "BB"]

In [65]:
with open(f'fg_params.yaml', "r") as f:
    fg_params: dict = yaml.safe_load(f)

### SO
args_so_lat = so_dict.so_dict()
args_so_lat.read_from_file('paramfiles/SO_MF_HF_args.dict')
args_so_lat['fsky'] = 0.56
args_so_lat['amp_dust'] = [fg_params['amp_dust']['SO'][args_so_lat['fsky']] for _ in args_so_lat['splits']]
args_so_lat['amp_sync'] = [fg_params['amp_sync']['SO'][args_so_lat['fsky']] for _ in args_so_lat['splits']]
args_so_lat['foregrounds'] = [['power_law', 'sync_power_law'] for _ in args_so_lat['splits']]

# add forecasts with galactic plane mask
args_so_lat_gp = deepcopy(args_so_lat)
args_so_lat_gp['fsky'] = 0.39
args_so_lat_gp['amp_dust'] = [fg_params['amp_dust']['SO'][args_so_lat_gp['fsky']] for _ in args_so_lat_gp['splits']]
args_so_lat_gp['amp_sync'] = [fg_params['amp_sync']['SO'][args_so_lat_gp['fsky']] for _ in args_so_lat_gp['splits']]


args_so_sat = so_dict.so_dict()
args_so_sat.read_from_file('paramfiles/SO_MF_HF_args.dict')
args_so_sat['fsky'] = 0.2
# TODO measure fg params in SAT patch ?
args_so_sat['amp_dust'] = [fg_params['amp_dust']['SO'][0.39] / 2 for _ in args_so_sat['splits']]
args_so_sat['amp_sync'] = [fg_params['amp_sync']['SO'][0.39] / 2 for _ in args_so_sat['splits']]
args_so_sat['foregrounds'] = [['power_law', 'sync_power_law'] for _ in args_so_sat['splits']]

### Planck
args_planck = so_dict.so_dict()
args_planck.read_from_file('paramfiles/Planck_HFI_args.dict')
args_planck['fsky'] = 0.92
args_planck['amp_dust'] = [fg_params['amp_dust']['full_sky'][args_planck['fsky']] for _ in args_planck['splits']]
args_planck['amp_sync'] = [fg_params['amp_sync']['full_sky'][args_planck['fsky']] for _ in args_planck['splits']]
args_planck['foregrounds'] = [['power_law', 'sync_power_law'] for _ in args_planck['splits']]

# add forecasts with galactic plane mask
args_planck_gp = deepcopy(args_planck)
args_planck_gp['fsky'] = 0.62
args_planck_gp['amp_dust'] = [fg_params['amp_dust']['full_sky'][args_planck_gp['fsky']] for _ in args_planck_gp['splits']]
args_planck_gp['amp_sync'] = [fg_params['amp_sync']['full_sky'][args_planck_gp['fsky']] for _ in args_planck_gp['splits']]


### LB
args_LB = so_dict.so_dict()
args_LB.read_from_file('paramfiles_old/LiteBIRD_args.dict')
args_LB['fsky'] = 0.92
args_LB['amp_dust'] = [fg_params['amp_dust']['full_sky'][args_LB['fsky']]  for _ in args_LB['splits']]
args_LB['amp_sync'] = [fg_params['amp_sync']['full_sky'][args_LB['fsky']]  for _ in args_LB['splits']]
args_LB['foregrounds'] = [['power_law', 'sync_power_law'] for _ in args_LB['splits']]

# add forecasts with galactic plane mask
args_LB_gp = deepcopy(args_LB)
args_LB_gp['fsky'] = 0.62
args_LB_gp['amp_dust'] = [fg_params['amp_dust']['full_sky'][args_LB_gp['fsky']] for _ in args_LB_gp['splits']]
args_LB_gp['amp_sync'] = [fg_params['amp_sync']['full_sky'][args_LB_gp['fsky']] for _ in args_LB_gp['splits']]


args_dict = {
    'SO LAT' : args_so_lat,
    'SO LAT GP' : args_so_lat_gp,
    'SO SAT' : args_so_sat,   # SAT TODO
    'Planck' : args_planck,
    'Planck GP' : args_planck_gp,
    'LiteBIRD' : args_LB,
    'LiteBIRD GP' : args_LB_gp,
}


In [67]:
sigma_beta = {}
sigma_alpha = {}
fisher_matrices = {}
sigma_stat = {}
for exp, args in args_dict.items():
    
    fisher_class = FisherBiref(**args)
    fisher_matrices[f'{exp}'] = fisher_class.fisher(0.0)
    cov_matrix = np.linalg.inv(fisher_matrices[f'{exp}']) * ((180 / pi) ** 2)
    sigma_beta[f'{exp}'] = np.sqrt(cov_matrix[0, 0])
    sigma_alpha[f'{exp}'] = np.sqrt(np.mean(cov_matrix[1:, 1:]))
    sigma_stat[f'{exp}'] = np.sqrt(1 / fisher_matrices[f'{exp}'][0, 0] * ((180 / pi) ** 2))
    print(f'{exp} : ±{sigma_beta[f"{exp}"]:.3f}degree')
print(' ')

SO LAT : ±0.055degree
SO LAT GP : ±0.156degree
SO SAT : ±0.363degree
Planck : ±0.119degree
Planck GP : ±0.244degree
LiteBIRD : ±0.044degree
LiteBIRD GP : ±0.071degree
 


In [73]:
from IPython.display import display, Markdown


latex_table = fr"""
\Large
\begin{{array}}{{|c|c|c|c|}}
\hline
& \sigma_\beta & \sigma_{{\alpha + \beta}} & \sigma_\alpha \\
\hline
\hline
Planck & {sigma_beta['Planck']:.3f} & {sigma_stat['Planck']:.3f} & {sigma_alpha['Planck']:.3f}  \\
\hline
SO\ LAT & {sigma_beta['SO LAT']:.3f} & {sigma_stat['SO LAT']:.3f} & {sigma_alpha['SO LAT']:.3f}  \\
\hline
SO\ SAT & {sigma_beta['SO SAT']:.3f} & {sigma_stat['SO SAT']:.3f} & {sigma_alpha['SO SAT']:.3f}  \\
\hline
LiteBIRD & {sigma_beta['LiteBIRD']:.3f} & {sigma_stat['LiteBIRD']:.3f} & {sigma_alpha['LiteBIRD']:.3f}  \\
\hline
\hline
Planck+GP & {sigma_beta['Planck GP']:.3f} & {sigma_stat['Planck GP']:.3f} & {sigma_alpha['Planck GP']:.3f}  \\
\hline
SO\ LAT+GP & {sigma_beta['SO LAT GP']:.3f} & {sigma_stat['SO LAT GP']:.3f} & {sigma_alpha['SO LAT GP']:.3f}  \\
\hline
LiteBIRD+GP & {sigma_beta['LiteBIRD GP']:.3f} & {sigma_stat['LiteBIRD GP']:.3f} & {sigma_alpha['LiteBIRD GP']:.3f}  \\
\hline
\hline
LOW FREQ EXPERIMENTS \\
\hline
\end{{array}}
"""


display(Markdown(f"$$ {latex_table} $$"))

$$ 
\Large
\begin{array}{|c|c|c|c|}
\hline
& \sigma_\beta & \sigma_{\alpha + \beta} & \sigma_\alpha \\
\hline
\hline
Planck & 0.119 & 0.043 & 0.104  \\
\hline
SO\ LAT & 0.055 & 0.005 & 0.052  \\
\hline
SO\ SAT & 0.363 & 0.008 & 0.360  \\
\hline
LiteBIRD & 0.044 & 0.006 & 0.043  \\
\hline
\hline
Planck+GP & 0.244 & 0.049 & 0.227  \\
\hline
SO\ LAT+GP & 0.156 & 0.006 & 0.154  \\
\hline
LiteBIRD+GP & 0.071 & 0.007 & 0.071  \\
\hline
\hline
LOW FREQ EXPERIMENTS \\
\hline
\end{array}
 $$